# Use Plotly for better Graphs in my Panel

- matplotlib - does not deal with long pathnames in the panel
    - works fine outside my dashboard - 


In [1]:
import pandas as pd
# use plotly over matplotlib
pd.options.plotting.backend = "plotly"


In [2]:
#df = pd.read_csv("./ws_s3_usage.csv", index_col = [0])
df = pd.read_csv("./ws_s3_usage.csv")

In [3]:
# https://www.stackvidhya.com/add-row-to-dataframe/
def groom_df(df, bucket):
    ndf = df.loc[df['bucket'] == bucket]
    ndf = ndf.sort_values(by=['Gbytes'],ascending=False)
    ndf = ndf.reset_index(drop=True)
    ndf = ndf.round()
    total = ndf[['Gbytes', 'Costs']].sum()
    total.name = 'myTotals'
    ndf = ndf.append(total)
    ndf.at[ndf.index[-1], 'DirPrefix'] = 'TOTAL'
    ndf.fillna('', inplace=True)
    
    return ndf
    

In [4]:
my_df = groom_df(df,'ws-in')

In [5]:
my_df

,bucket,DirPrefix,Gbytes,Costs
0,ws-in,NorthAmerica/,2361.0,54.0
1,ws-in,CONUS/,1153.0,27.0
2,ws-in,DelawareRiverBasin/,698.0,16.0
3,ws-in,cfactor/,150.0,3.0
4,ws-in,EastAfrica/,26.0,1.0
5,ws-in,home/,9.0,0.0
6,ws-in,global_dT/,4.0,0.0
7,ws-in,Marshall_Islands/,3.0,0.0
myTotals,,TOTAL,4404.0,101.0


In [6]:
cost_total = my_df.iloc[-1]['Costs']

In [7]:
cost_total

101.0

In [8]:
import panel as pn
#pn.extension()
pn.extension('plotly')

In [9]:
title = '# Fun with Buckets'

In [10]:
def get_bucket_list():
    '''
    ___________________________________________________________________
    THIS function returns watersmart S3 bucket names
    __________________________________________________________________
      - we only need THREE (3) buckets to do our work
    our buckets are used for our data inputs
          - ws-in: holds precip, NDVI, Temperature - inputs to our water models
          - ws-out: is our model outputs - sometimes just tiles and chips
          - ws-enduser: - is where we mosaic these into usable tifs, netcdfs, ugly-tarballs 
          ... etc ...
    '''
    b_list = ['ws-in', 'ws-out', 'ws-enduser']
    return b_list

In [11]:
bucket_select = pn.widgets.Select(name='IMPORTANT_BUCKET_LIST', options=get_bucket_list())

In [12]:
table_plot_select = pn.widgets.RadioBoxGroup(name='RadioBoxGroup', options=['Plot','Table'], inline=True)


In [13]:
display_select = pn.widgets.MultiSelect(width_policy='min', name='Display Select', value=['Costs',],
    options=['Gbytes', 'Costs',], size=2)


In [14]:
selectors = pn.Row(bucket_select, display_select, table_plot_select)

In [15]:
import matplotlib.pyplot as plt

def return_plot_table_please(bucket, plot):
    my_df = groom_df(df,bucket)
    print(plot)
    df_widget = pn.widgets.DataFrame(my_df, show_index=True, 
        fit_columns=True, row_height=23, name='Bucket Usage Table')
    iaxs = my_df.plot.bar(x='DirPrefix', y='Costs', title=f'BUCKET:{bucket}', width=500, height=400)
    
    costs_total = my_df.iloc[-1]['Costs']
    usage_guage.value = costs_total
    
    if 'Plot' in plot:
        return(iaxs)
    else:
        return(df_widget)

In [16]:
@pn.depends(bucket_select.param.value)
def refresh_plot_table(bucket_select):
    plot_v_table = table_plot_select.value
    return(return_plot_table_please(bucket_select, plot_v_table))

In [17]:
usage_guage = pn.indicators.Dial(name='Budget', value=900, bounds=(0, 1300), height=100, width=100, 
                            format='{value} dollars', 
                            colors = [(0.2, 'green'), (0.8, 'gold'), (1, 'red')],
                            margin=(100,10))

In [18]:
my_layout = pn.Column( 
    title,
    selectors,
    pn.Row(refresh_plot_table, usage_guage),
    '### the end'
)

Plot


In [19]:
my_layout.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Select(name='IMPORTANT_BUCKET_LIST', options=['ws-in', 'ws-out', ...], value='ws-in')
        [1] MultiSelect(name='Display Select', options=['Gbytes', 'Costs'], size=2, value=['Costs'], width_policy='min')
        [2] RadioBoxGroup(inline=True, options=['Plot', 'Table'], value='Plot')
    [2] Row
        [0] ParamFunction(function)
        [1] Dial(bounds=(0, 1300), colors=[(0.2, 'green'), ...], format='{value} dollars', height=100, margin=(100, 10), name='Budget', value=101.0, width=100)
    [3] Markdown(str)

## https://panel.holoviz.org/reference/widgets/MultiSelect.html